In [1]:
# code to access and manipulate data from the 
# FCC Database: http://www.fcc.gov/uls/transactions/daily-weekly#fcc-uls-transaction-files-weekly

#references: 
# - https://www.ae7q.com/query/
# - https://www.fcc.gov/reports-research/developers/license-view-api

In [2]:
#---------------imports and setup
import requests
import pandas as pd
import numpy as np
import openpyxl
import zipfile
import io


#setup logging <- setup needs to be specific to jupyter notebook
import logging

#we do this to set up logging so it works in jupyter notebook AND does not spawn duplicate loggers if the cell is run repeatedly
try:
    logger
    
except NameError: #logging not set up yet
    
    logger = logging.getLogger()
    fhandler = logging.FileHandler(filename='python.log', mode='a')
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    fhandler.setFormatter(formatter)
    logger.addHandler(fhandler)
    logger.setLevel(logging.DEBUG)
    
logging.info(f'starting to log for FCC_data')

In [3]:
#----------------Jupyter Notebook Helpers

# display settings 
#see https://stackoverflow.com/questions/21971449/how-do-i-increase-the-cell-width-of-the-jupyter-ipython-notebook-in-my-browser

#display full window width
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


In [4]:
#-------------------------API Endpoints

# https://www.ae7q.com/query/ database query: https://www.ae7q.com/query/stat/DataBase.php?SCHEMA=Lic

fcc_raw_amateur_licence_data_download_url = "https://data.fcc.gov/download/pub/uls/complete/l_amat.zip"
# field definitions at https://www.fcc.gov/sites/default/files/public_access_database_definitions_v10_0.pdf


In [5]:
#-----------------------Get FCC Licence Database

logging.info(f'starting to download fcc data')
r = requests.get(fcc_raw_amateur_licence_data_download_url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall(".")


In [6]:
# ---------------------import the Database into DataFrame

logging.info(f'loading downloaded fcc data')
df_fcc_en = pd.read_csv('./EN.dat', sep='|', index_col=False,
                            names=[ "Record Type [EN]",
                                    "Unique System Identifier",
                                    "ULS File Number",
                                    "EBF Number",
                                    "Call Sign",
                                    "Entity Type",
                                    "Licensee ID",
                                    "Entity Name",
                                    "First Name",
                                    "MI",
                                    "Last Name",
                                    "Suffix",
                                    "Phone",
                                    "Fax",
                                    "Email",
                                    "Street Address",
                                    "City",
                                    "State",
                                    "Zip Code",
                                    "PO Box",
                                    "Attention Line",
                                    "SGIN",
                                    "FCC Registration Number (FRN)",
                                    "Applicant Type Code",
                                    "Applicant Type Code Other",
                                    "Status Code",
                                    "Status Date",
                                    "3.7 GHz License Type",
                                    "Linked Unique System Identifier"
                                    "Linked Call Sign"
                                  ])

logging.info(f'fcc data loaded into df_fcc_en')

/tmp/ipykernel_35612/4008298916.py:4: DtypeWarning: Columns (12,18,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df_fcc_en = pd.read_csv('./EN.dat', sep='|', index_col=False,


In [7]:
# -----------------------POTA Data

logging.info(f'starting to download POTA data')

pota_activators_api_url = "https://api.pota.app/activator/all?year=2023" 
df_pota_activators = pd.read_json(pota_activators_api_url)
df_pota_activators = df_pota_activators.add_prefix('pota_activator_')
df_pota_activators['pota_activator_flag'] = True

pota_hunters_api_url = "https://api.pota.app/hunter/all?year=2023" 
df_pota_hunters = pd.read_json(pota_hunters_api_url)
df_pota_hunters = df_pota_hunters.add_prefix('pota_hunter_')
df_pota_hunters['pota_hunter_flag'] = True

logging.info(f'POTA data downloaded')

In [8]:
#-------------------------Database joins

logging.info(f'joining FCC Data and POTA data')

df_consolidated_HamPota_Data_set = df_fcc_en.merge(df_pota_activators, left_on='Call Sign', right_on='pota_activator_activeCallsign', how='left')
df_consolidated_HamPota_Data_set = df_consolidated_HamPota_Data_set.merge(df_pota_hunters, left_on='Call Sign', right_on='pota_hunter_activeCallsign', how='left')



In [9]:
#---------------------role tagging

logging.info(f'generating Ham/Activator/Hunter consolidated role tag')

df_consolidated_HamPota_Data_set = df_consolidated_HamPota_Data_set.assign(HamPotaRoleFlag="")

for row in df_consolidated_HamPota_Data_set.index :
    consolidatedRole = "Ham"
    if df_consolidated_HamPota_Data_set["pota_activator_flag"][row] == True : consolidatedRole = consolidatedRole + "Activator"
    if df_consolidated_HamPota_Data_set["pota_hunter_flag"][row] == True : consolidatedRole = consolidatedRole + "Hunter"
    df_consolidated_HamPota_Data_set['HamPotaRoleFlag'][row] = consolidatedRole

/tmp/ipykernel_35612/2733699581.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_consolidated_HamPota_Data_set['HamPotaRoleFlag'][row] = consolidatedRole


In [10]:
df_consolidated_HamPota_Data_set

,Record Type [EN],Unique System Identifier,ULS File Number,EBF Number,Call Sign,Entity Type,Licensee ID,Entity Name,First Name,MI,...,pota_activator_TotalPhoneContacts,pota_activator_flag,pota_hunter_activeCallsign,pota_hunter_numParks,pota_hunter_numQSOs,pota_hunter_qsosCW,pota_hunter_qsosDATA,pota_hunter_qsosPHONE,pota_hunter_flag,HamPotaRoleFlag
0,EN,215000,NaN,NaN,AA0A,L,L00209566,"MC CARTHY, DENNIS J",DENNIS,J,...,0.0,True,AA0A,7.0,7.0,0.0,7.0,0.0,True,HamActivatorHunter
1,EN,215001,NaN,NaN,AA0AA,L,L00196154,"MONKS, WILLIAM S",WILLIAM,S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ham
2,EN,215002,NaN,NaN,AA0AB,L,L00185374,"CROM SR, RAYMOND L",RAYMOND,L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ham
3,EN,215003,NaN,NaN,AA0AC,L,NaN,"PETH, ESTHER T",ESTHER,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ham
4,EN,215004,NaN,NaN,AA0AD,L,L00310459,"Odermann, William",William,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ham
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1580736,EN,4775115,NaN,NaN,KC1TAQ,L,L02675406,"Bonaccorsi, Matthew",Matthew,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ham
1580737,EN,4775116,NaN,NaN,KC1TAR,L,L02675407,"TURNER JR, TIMOTHY R",TIMOTHY,R,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ham
1580738,EN,4775117,NaN,NaN,KF0MXO,L,L02675408,"RAMIREZ, ERIC L",ERIC,L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ham
1580739,EN,4775118,NaN,NaN,KJ5BIK,L,L02675409,"SMITH, BRYAN T",BRYAN,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ham


In [13]:
#------------persistence

#persist to an excel file
file_save_location = "FCC_POTA_USA_HAM_Activator_Hunter.csv"
df_consolidated_HamPota_Data_set.to_csv(file_save_location)
logging.info(f'data persisted to {file_save_location}')
